In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator
from sklearn.model_selection import cross_val_score


from functions import *

pd.set_option('display.max_columns', 100)

# load

In [16]:
app_train = pd.read_csv('processed-data/processed_application_train.csv')
app_test = pd.read_csv('processed-data/processed_application_test.csv')

In [17]:
app_train.set_index('SK_ID_CURR', inplace=True)
app_test.set_index('SK_ID_CURR', inplace=True)

In [18]:
app_train.head()

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,FLOORSMAX_MODE,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_8,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_nan,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_nan,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,FONDKAPREMONT_MODE_nan,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,DOCUMENT_COUNT,NEW_DOC_KURT,AGE_RANGE
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
278621,0,0,0,0,0,1,0,1,0,2.0,1,1,0,0,0,0,0,0,0.2917,0.0,0.0,1,0,0,0,0.0,0.0,0.0,0.0,True,False,False,True,False,False,False,False,True,True,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,False,1,20.0,3
139008,0,0,0,1,0,1,0,0,0,2.0,2,2,0,0,0,0,0,0,NaN,0.0,0.0,1,0,0,0,NaN,NaN,NaN,NaN,True,False,False,False,False,True,False,False,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,1,20.0,4
138348,0,1,0,1,0,1,0,0,0,1.0,2,2,0,0,0,0,1,1,NaN,0.0,0.0,0,0,0,1,0.0,0.0,0.0,0.0,True,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,True,False,False,False,False,False,False,True,F

# merge

In [19]:
credit_card = pd.read_csv('processed-data/processed_credit_card_balance.csv')
credit_card.set_index('SK_ID_CURR', inplace=True)
credit_card.head()

,CC_SK_ID_PREV_MIN,CC_SK_ID_PREV_MAX,CC_SK_ID_PREV_MEAN,CC_SK_ID_PREV_SUM,CC_SK_ID_PREV_VAR,CC_MONTHS_BALANCE_MIN,CC_MONTHS_BALANCE_MAX,CC_MONTHS_BALANCE_MEAN,CC_MONTHS_BALANCE_SUM,CC_MONTHS_BALANCE_VAR,CC_AMT_BALANCE_MIN,CC_AMT_BALANCE_MAX,CC_AMT_BALANCE_MEAN,CC_AMT_BALANCE_SUM,CC_AMT_BALANCE_VAR,CC_AMT_CREDIT_LIMIT_ACTUAL_MIN,CC_AMT_CREDIT_LIMIT_ACTUAL_MAX,CC_AMT_CREDIT_LIMIT_ACTUAL_MEAN,CC_AMT_CREDIT_LIMIT_ACTUAL_SUM,CC_AMT_CREDIT_LIMIT_ACTUAL_VAR,CC_AMT_DRAWINGS_ATM_CURRENT_MIN,CC_AMT_DRAWINGS_ATM_CURRENT_MAX,CC_AMT_DRAWINGS_ATM_CURRENT_MEAN,CC_AMT_DRAWINGS_ATM_CURRENT_SUM,CC_AMT_DRAWINGS_ATM_CURRENT_VAR,CC_AMT_DRAWINGS_CURRENT_MIN,CC_AMT_DRAWINGS_CURRENT_MAX,CC_AMT_DRAWINGS_CURRENT_MEAN,CC_AMT_DRAWINGS_CURRENT_SUM,CC_AMT_DRAWINGS_CURRENT_VAR,CC_AMT_DRAWINGS_OTHER_CURRENT_MIN,CC_AMT_DRAWINGS_OTHER_CURRENT_MAX,CC_AMT_DRAWINGS_OTHER_CURRENT_MEAN,CC_AMT_DRAWINGS_OTHER_CURRENT_SUM,CC_AMT_DRAWINGS_OTHER_CURRENT_VAR,CC_AMT_DRAWINGS_POS_CURRENT_MIN,CC_AMT_DRAWINGS_POS_CURRENT_MAX,CC_AMT_DRAWINGS_POS_CURRENT_MEAN,CC_AMT_DRAWINGS_POS_CURRENT_SUM,CC_AMT_DRAWINGS_POS_CURRENT_VAR,CC_AMT_INST_MIN_REGULARITY_MIN,CC_AMT_INST_MIN_REGULARITY_MAX,CC_AMT_INST_MIN_REGULARITY_MEAN,CC_AMT_INST_MIN_REGULARITY_SUM,CC_AMT_INST_MIN_REGULARITY_VAR,CC_AMT_PAYMENT_CURRENT_MIN,CC_AMT_PAYMENT_CURRENT_MAX,CC_AMT_PAYMENT_CURRENT_MEAN,CC_AMT_PAYMENT_CURRENT_SUM,CC_AMT_PAYMENT_CURRENT_VAR,...,INS_24M_CNT_DRAWINGS_CURRENT_MAX,INS_24M_CNT_DRAWINGS_CURRENT_MEAN,INS_24M_CNT_DRAWINGS_CURRENT_SUM,INS_24M_CNT_DRAWINGS_POS_CURRENT_MEAN,INS_24M_SK_DPD_MEAN,INS_24M_SK_DPD_MAX,INS_24M_SK_DPD_SUM,INS_24M_DRAWING_LIMIT_RATIO_MIN,INS_24M_DRAWING_LIMIT_RATIO_MAX,INS_24M_LATE_PAYMENT_MEAN,INS_24M_LATE_PAYMENT_SUM,INS_24M_CARD_IS_DPD_UNDER_120_MEAN,INS_24M_CARD_IS_DPD_UNDER_120_SUM,INS_24M_CARD_IS_DPD_OVER_120_MEAN,INS_24M_CARD_IS_DPD_OVER_120_SUM,INS_48M_AMT_BALANCE_MEAN,INS_48M_AMT_BALANCE_MAX,INS_48M_LIMIT_USE_MIN,INS_48M_LIMIT_USE_MAX,INS_48M_AMT_CREDIT_LIMIT_ACTUAL_MAX,INS_48M_AMT_DRAWINGS_ATM_CURRENT_MAX,INS_48M_AMT_DRAWINGS_ATM_CURRENT_SUM,INS_48M_AMT_DRAWINGS_CURRENT_MAX,INS_48M_AMT_DRAWINGS_CURRENT_SUM,INS_48M_AMT_DRAWINGS_POS_CURRENT_MAX,INS_48M_AMT_DRAWINGS_POS_CURRENT_SUM,INS_48M_AMT_INST_MIN_REGULARITY_MAX,INS_48M_AMT_INST_MIN_REGULARITY_MEAN,INS_48M_AMT_PAYMENT_TOTAL_CURRENT_MAX,INS_48M_AMT_PAYMENT_TOTAL_CURRENT_SUM,INS_48M_AMT_TOTAL_RECEIVABLE_MAX,INS_48M_AMT_TOTAL_RECEIVABLE_MEAN,INS_48M_CNT_DRAWINGS_ATM_CURRENT_MAX,INS_48M_CNT_DRAWINGS_ATM_CURRENT_SUM,INS_48M_CNT_DRAWINGS_ATM_CURRENT_MEAN,INS_48M_CNT_DRAWINGS_CURRENT_MAX,INS_48M_CNT_DRAWINGS_CURRENT_MEAN,INS_48M_CNT_DRAWINGS_CURRENT_SUM,INS_48M_CNT_DRAWINGS_POS_CURRENT_MEAN,INS_48M_SK_DPD_MEAN,INS_48M_SK_DPD_MAX,INS_48M_SK_DPD_SUM,INS_48M_DRAWING_LIMIT_RATIO_MIN,INS_48M_DRAWING_LIMIT_RATIO_MAX,INS_48M_LATE_PAYMENT_MEAN,INS_48M_LATE_PAYMENT_SUM,INS_48M_CARD_IS_DPD_UNDER_120_MEAN,INS_48M_CARD_IS_DPD_UNDER_120_SUM,INS_48M_CARD_IS_DPD_OVER_120_MEAN,INS_48M_CARD_IS_DPD_OVER_120_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1745718,1745718,1745718.0,13965744,0.0,-8,-1,-4.5,-36,6.0,0.0,36569.430,6735.504375,53884.035,1.655502e+08,180000,180000,180000.000000,1440000,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000e+00,0.0,56187.00,14053.455000,112427.640,4.023188e+08,0.0,0.0,0.0,0.0,0.0,0.0,56187.00,14053.4550,112427.640,4.023188e+08,0.0,2250.000,281.250000,2250.000,6.328125e+05,488.07,63574.920,14310.649286,100174.545,5.097456e+08,...,10,2.625000,21,2.625,0.0,0,0,0.0,0.000,0.0,0,0.0,0,0.0,0,6735.504375,36569.430,0.000000,0.203163,180000,0.0,0.0,56187.00,112427.640,56187.00,112427.640,2250.000,281.250000,63574.920,95674.545,36569.43,6735.504375,0.0,0.0,0.000000,10,2.625000,21,2.625,0.0,0,0,0.0,0.000,0.0,0,0.0,0,0.0,0
1,1723965,1723965,1723965.0,15515685,0.0,-9,-1,-5.0,-45,7.5,15271.2,227526.210,147180.880000,1324627.920,9.632267e+09,225000,225000,225000.000000,2025000,0.000000e+00,0.0,180000.0,26000.000000,234000.0,3.431250e+09,0.0,180000.00,26000.00

In [20]:
# merge credit card data
app_train = app_train.merge(credit_card, left_index=True, right_index=True, how='left')
app_test = app_test.merge(credit_card, left_index=True, right_index=True, how='left')

# train

In [21]:
X, y = app_train.drop(['TARGET'], axis=1), app_train['TARGET']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

# replace inf values with nan
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_valid = X_valid.replace([np.inf, -np.inf], np.nan)

# robust scaler
robust_scaler = RobustScaler(quantile_range=(0, 98))
robust_scaler.fit(X_train)
X_train = robust_scaler.transform(X_train)
X_valid = robust_scaler.transform(X_valid)

# scale values
minmax_scaler = MinMaxScaler(feature_range=(0, 1))
minmax_scaler.fit(X_train)
X_train = minmax_scaler.transform(X_train)
X_valid = minmax_scaler.transform(X_valid)

In [22]:
# fill missing values with median
imputer = SimpleImputer(strategy='mean')
imputer.fit(X_train)
X_train = imputer.transform(X_train)
X_valid = imputer.transform(X_valid)

In [23]:
# train model
logreg = LogisticRegression(class_weight='balanced', solver='newton-cholesky', C=0.01)

# cross validation
scores = cross_val_score(logreg, X_train, y_train, cv=5, scoring='roc_auc')
gini = 2 * scores.mean() - 1

print('GINI: {:.4f}'.format(gini))

GINI: 0.3471


In [24]:
# valid on X_valid
logreg.fit(X_train, y_train)
y_pred = logreg.predict_proba(X_valid)[:, 1]

print('GINI: {:.4f}'.format(2*roc_auc_score(y_valid, y_pred)-1))

GINI: 0.3470


# test

In [25]:
# reorder columns to match train
app_test_clean = app_test[X.columns]

# replace inf values with nan
X_test = app_test_clean.replace([np.inf, -np.inf], np.nan)

# robust scaler
X_test = robust_scaler.transform(X_test)

# scale values
X_test = minmax_scaler.transform(X_test)

# fill missing values with median
X_test = imputer.transform(X_test)

# predict
y_pred = logreg.predict_proba(X_test)[:, 1]

# submission
submission = pd.DataFrame({'SK_ID_CURR': app_test_clean.index, 'TARGET': y_pred})

submission.head()

,SK_ID_CURR,TARGET
0,83659,0.577165
1,174814,0.578110
2,179486,0.649131
3,57038,0.260366
4,25672,0.356092


In [26]:
# set index
submission.set_index('SK_ID_CURR', inplace=True)
submission.sort_index(inplace=True)

# save
submission.to_csv('submission.csv')
submission.head()

,TARGET
SK_ID_CURR,
5,0.502404
13,0.630309
16,0.692085
19,0.564358
25,0.553866
